# Introduction

Today we'll dive deep into a dataset all about LEGO. From the dataset we can ask whole bunch of interesting questions about the history of the LEGO company, their product offering, and which LEGO set ultimately rules them all:

<ul type="square">
<li>What is the most enormous LEGO set ever created and how many parts did it have?</li>

<li>How did the LEGO company start out? In which year were the first LEGO sets released and how many sets did the company sell when it first launched?</li>

<li>Which LEGO theme has the most sets? Is it one of LEGO's own themes like Ninjago or a theme they licensed liked Harry Potter or Marvel Superheroes?</li>

<li>When did the LEGO company really expand its product offering? Can we spot a change in the company strategy based on how many themes and sets did it released year-on-year?</li>

<li>Did LEGO sets grow in size and complexity over time? Do older LEGO 
sets tend to have more or fewer parts than newer sets?</li>
</ul>

**Data Source**

[Rebrickable](https://rebrickable.com/downloads/) has compiled data on all the LEGO pieces in existence. I recommend you use download the .csv files provided in this lesson. 

<img src="assets/bricks.jpg" />

# Import Statements

In [ ]:
!pip install "polars[all]"

# Data Exploration

**Challenge**: How many different colours does the LEGO company produce? Read the colors.csv file in the data folder and find the total number of unique colours. Try using the [.nunique() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html?highlight=nunique#pandas.DataFrame.nunique) to accomplish this.

In [ ]:
import polars as pl

In [ ]:
colors = pl.read_csv("data/colors.csv")

In [ ]:
len(colors)

**Challenge**: Find the number of transparent colours where <code>is_trans == 't'</code> versus the number of opaque colours where <code>is_trans == 'f'</code>. See if you can accomplish this in two different ways.

In [ ]:
colors.head()

In [ ]:
colors.group_by('is_trans').len()

**Challenge**: Change this into an h3 section heading: Understanding LEGO Themes vs. LEGO Sets

### Understanding LEGO Themes vs. LEGO Sets

Walk into a LEGO store and you will see their products organised by theme. Their themes include Star Wars, Batman, Harry Potter and many more.

**Challenge**: Display this image: https://i.imgur.com/aKcwkSx.png

Walk into a LEGO store and you will see their products organised by theme. Their themes include Star Wars, Batman, Harry Potter and many more.

<img src="https://i.imgur.com/aKcwkSx.png" >

A lego set is a particular box of LEGO or product. Therefore, a single theme typically has many different sets.

<img src="https://i.imgur.com/whB1olq.png" />

The <code>sets.csv</code> data contains a list of sets over the years and the number of parts that each of these sets contained.

**Challenge**: Read the sets.csv data and take a look at the first and last couple of rows. 

In [ ]:
sets = pl.read_csv("data/sets.csv")

In [ ]:
sets.head()

In [ ]:
sets.tail()

**Challenge**: In which year were the first LEGO sets released and what were these sets called?

In [ ]:
first_set = sets.sort('year')[0]
first_set_year = first_set['year'].item()
first_set_name = first_set['name'].item()
print(f"First LEGO released in {first_set_year} and were called {first_set_name}")

**Challenge**: How many different sets did LEGO sell in their first year? How many types of LEGO products were on offer in the year the company started?

In [ ]:
first_year_sets = sets.filter(pl.col('year') == first_set_year)
first_year_sets_len = len(first_year_sets)
first_year_themes = len(first_year_sets.group_by('theme_id').agg())

**Challenge**: Find the top 5 LEGO sets with the most number of parts. 

In [ ]:
largest_lego_sets = sets.sort('num_parts').tail()

**Challenge**: Use <code>.groupby()</code> and <code>.count()</code> to show the number of LEGO sets released year-on-year. How do the number of sets released in 1955 compare to the number of sets released in 2019? 

In [ ]:
sets_per_year_sorted = sets.group_by('year').len().sort('year')
sets_in_1955 = sets_per_year_sorted.row(by_predicate=pl.col('year') == 1955)[1]
sets_in_2019 = sets_per_year_sorted.row(by_predicate=pl.col('year') == 2019)[1]
print(f"1955 {sets_in_1955} vs 2019 {sets_in_2019}")

**Challenge**: Show the number of LEGO releases on a line chart using Matplotlib. <br>
<br>
Note that the .csv file is from late 2020, so to plot the full calendar years, you will have to exclude some data from your chart. Can you use the slicing techniques covered in Day 21 to avoid plotting the last two years? The same syntax will work on Pandas DataFrames. 

In [ ]:
sets_by_year = sets.group_by('year').len().sort('year')[:-2]
sets_by_year

In [ ]:
sets_by_year.plot(x='year', y='len')

### Aggregate Data with the Python .agg() Function

Let's work out the number of different themes shipped by year. This means we have to count the number of unique theme_ids per calendar year.

In [ ]:
themes_by_year = sets.sort('year').group_by('year').agg(pl.col('theme_id').n_unique())[:-2].rename({'theme_id': 'num_themes'})

In [ ]:
themes_by_year.head()

In [ ]:
themes_by_year.tail()

**Challenge**: Plot the number of themes released by year on a line chart. Only include the full calendar years (i.e., exclude 2020 and 2021). 

In [ ]:
themes_by_year.plot(x='year', y='num_themes')

### Line Charts with Two Seperate Axes

In [ ]:
import hvplot.polars

In [ ]:
sets.sort('year').group_by('year').len().rename({"len": "num_sets"})

In [ ]:
sets_by_year = sets.sort('year').group_by('year').len()[:-2].rename({"len": "num_sets"})
sets_by_year.plot(x='year', y='num_sets')

In [ ]:
hvplot.extension('bokeh')
plot1 = themes_by_year.plot(x='year', y='num_themes')
plot2 = sets_by_year.plot(x='year', y='num_sets')
(plot1 * plot2).opts(multi_y=True)

**Challenge**: Use the <code>.groupby()</code> and <code>.agg()</code> function together to figure out the average number of parts per set. How many parts did the average LEGO set released in 1954 compared to say, 2017?

In [ ]:
parts_per_set = sets.sort('year').group_by('year').agg(pl.col('num_parts').mean())[:-2]
parts_per_set_1954 = parts_per_set.row(by_predicate=pl.col('year') == 1954)[1]
parts_per_set_2017 = parts_per_set.row(by_predicate=pl.col('year') == 2017)[1]
print(f"The average LEGO set in 1954 had {parts_per_set_1954}, compared to the average of {parts_per_set_2017} in 2017")

In [ ]:
parts_per_set.tail()

### Scatter Plots in Matplotlib

**Challenge**: Has the size and complexity of LEGO sets increased over time based on the number of parts? Plot the average number of parts over time using a Matplotlib scatter plot. See if you can use the [scatter plot documentation](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.scatter.html) before I show you the solution. Do you spot a trend in the chart? 

In [ ]:
parts_per_set.plot.scatter(x='year', y='num_parts')

### Number of Sets per LEGO Theme

LEGO has licensed many hit franchises from Harry Potter to Marvel Super Heros to many others. But which theme has the largest number of individual sets? 

In [ ]:
sets['theme_id'].value_counts()

**Challenge** Use what you know about HTML markup and tags to display the database schema: https://i.imgur.com/Sg4lcjx.png

<img src="https://i.imgur.com/Sg4lcjx.png">

### Database Schemas, Foreign Keys and Merging DataFrames

The themes.csv file has the actual theme names. The sets .csv has <code>theme_ids</code> which link to the <code>id</code> column in the themes.csv. 

**Challenge**: Explore the themes.csv. How is it structured? Search for the name 'Star Wars'. How many <code>id</code>s correspond to this name in the themes.csv? Now use these <code>id</code>s and find the corresponding the sets in the sets.csv (Hint: you'll need to look for matches in the <code>theme_id</code> column)

In [ ]:
themes = pl.read_csv('data/themes.csv')

In [ ]:
star_wars_theme_ids = themes.filter(name='Star Wars')

In [ ]:
for theme_id in star_wars_theme_ids.iter_rows():
    id = theme_id[0]
    result = sets.filter(pl.col('theme_id') == id)
    print(result)

In [ ]:
star_wars_theme_ids

In [ ]:
sets.filter(pl.col('theme_id').is_in(star_wars_theme_ids['id']))

### Merging (i.e., Combining) DataFrames based on a Key


In [ ]:
set_theme_count = sets['theme_id'].value_counts()

In [ ]:
#set_theme_count = set_theme_count.sort('count', descending=True)

In [ ]:
set_theme_count = set_theme_count.rename({"theme_id": "id"})

In [ ]:
set_theme_count

In [ ]:
themes.join(set_theme_count, on='id').sort('count', descending=True).head(10).plot.bar(x='name', y='count')

In [ ]:
set_theme_count = sets['theme_id'].value_counts()

In [ ]:
set_theme_count

In [ ]:
themes.join(set_theme_count, left_on='id', right_on='theme_id').sort('count', descending=True).head(10).plot.bar(x='name', y='count')